In [ ]:
# %pip install memory_profiler

In [1]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

# from folium import LayerControl
from eo_tools.util import show_insar_coh, show_insar_phi, show_sar_int
import geopandas as gpd
from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:eodag.config:Loading user configuration from: /data/eodag_config.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /root/.config/eodag/locations.yml


# Download S-1 products

In [2]:
# load a geometry
file_aoi = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
ids = [
 "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
 "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
]
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-03",
    "end": "2023-09-17",
    "geom": shp
}

results, _ = dag.search(**search_criteria)
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
# dag.download_all(to_dl, outputs_prefix="/data/S1/", extract=False)

INFO:eodag.core:Searching product type 'S1_SAR_SLC' on provider: cop_dataspace
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?startDate=2023-09-03&completionDate=2023-09-17&geometry=POLYGON ((-9.0113 30.7107, -9.0113 31.2571, -8.1129 31.2571, -8.1129 30.7107, -9.0113 30.7107))&productType=SLC&maxRecords=20&page=1&exactCount=1
INFO:eodag.core:Found 6 result(s) on provider 'cop_dataspace'


2 products to download


# Set up parameters and output dir

## Important
- The products must be **unzipped** either manually or by using `extract=True` in the download call. 
- In the following products, only IW1 and IW2 are relevant as IW3 contains only water pixels.

In [3]:
data_dir = "/data/S1"
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
out_dir = f"/data/res/refactor-proc-dask"
iw = 1 # subswath
pol = "vv" # polarization ("vv"or "vh")
min_burst = 5
max_burst = 5 # Set to None to process all (warning: memory hungry)

# Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import preprocess_insar_iw

# TODO: use downloaded products
preprocess_insar_iw(
    primary_dir,
    secondary_dir,
    out_dir,
    iw=iw,
    pol=pol,
    min_burst=min_burst,
    max_burst=max_burst,
    dem_upsampling=1.8,
    apply_fast_esd=True,
    # dem_force_download=True
)

# Apply geocoding using lookup tables

In [9]:
from eo_tools.S1.process import slc2geo, coherence, amplitude
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_amp = f"{out_dir}/amp.tif"
file_coh = f"{out_dir}/coh.tif"
file_phi_geo = f"{out_dir}/phi_geo.tif"
file_amp_geo = f"{out_dir}/amp_geo.tif"
file_coh_geo = f"{out_dir}/coh_geo.tif"
file_lut = f"{out_dir}/lut.tif"
# computing amplitude and complex coherence  in the radar geometry
# coherence(file_prm, file_sec, file_coh, box_size=[3,10], magnitude=False)
# amplitude(file_prm, file_amp)

# combined multilooking and geocoding
# interferometric coherence
# slc2geo(file_coh, file_lut, file_coh_geo, 1, 4, 3, write_phase=False, magnitude_only=True)

# interferometric phase
# slc2geo(file_coh, file_lut, file_phi_geo, 1, 4, 3, write_phase=True)

# amplitude of the primary image
slc2geo(file_amp, file_lut, file_amp_geo, 1, 4, 3, False, True)

INFO:eo_tools.S1.process:Project image with the lookup table.


# Visualize outputs on a folium map

In [16]:
show_sar_int(file_amp_geo)

INFO:httpx:HTTP Request: GET http://localhost:8085/cog/statistics?url=%2Fdata%2Fres%2Frefactor-proc-dask%2Famp_geo.tif "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8085/cog/info?url=%2Fdata%2Fres%2Frefactor-proc-dask%2Famp_geo.tif "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8085/cog/tilejson.json?url=%2Fdata%2Fres%2Frefactor-proc-dask%2Famp_geo.tif&rescale=0.142122283577919%2C2.29915452003479&resampling=average "HTTP/1.1 200 OK"


In [ ]:
show_insar_phi(file_phi_geo)

In [ ]:
show_insar_coh(file_coh_geo)